In [ ]:
from Oracle_Assignment_2 import q3_linear_1, q3_linear_2, q3_stocknet

In [ ]:
Q3_linear_1_data = q3_linear_1(23684)
Q3_linear_2_data = q3_linear_2(23684)
Q3_stocknet_data = q3_stocknet(23684)

In [ ]:
print(Q3_stocknet_data)

In [ ]:
import numpy as np

def ols_regression(X, y):
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

def ridge_regression(X, y, lambda_=1.0):
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    
    n_features = X.shape[1]
    w = np.linalg.inv(X.T @ X + lambda_ * np.eye(n_features)) @ X.T @ y
    return w

def predict(X, w):
    X = np.hstack([np.ones((X.shape[0], 1)), X]) # Adding bias(1) term
    return X @ w
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

In [ ]:
print(len(Q3_linear_1_data))
D1_X_train,D1_y_train,D1_X_test,D1_y_test = Q3_linear_1_data

D1_X_train = np.array(D1_X_train)
D1_y_train = np.array(D1_y_train)
D1_X_test = np.array(D1_X_test)
D1_y_test = np.array(D1_y_test)

D2_X_train,D2_y_train,D2_X_test,D2_y_test = Q3_linear_2_data

D2_X_train = np.array(D2_X_train)
D2_y_train = np.array(D2_y_train)
D2_X_test = np.array(D2_X_test)
D2_y_test = np.array(D2_y_test)

print(D1_X_train.shape)
print(D1_y_train.shape)
print(D1_X_test.shape)
print(D1_y_test.shape)

In [ ]:
w_1_ols = ols_regression(D1_X_train, D1_y_train)
w_1_rr = ridge_regression(D1_X_train, D1_y_train)

w_2_ols = ols_regression(D2_X_train, D2_y_train)
w_2_rr = ridge_regression(D2_X_train, D2_y_train)

In [ ]:
np.savetxt("w_ols_23684.csv", w_2_ols, delimiter = ",")
np.savetxt("w_rr_23684.csv", w_2_rr, delimiter = ",")

In [ ]:
# mean squared error

y_pred_1_ols = predict(D1_X_test, w_1_ols)
y_pred_1_rr = predict(D1_X_test, w_1_rr)

y_pred_2_ols = predict(D2_X_test, w_2_ols)
y_pred_2_rr = predict(D2_X_test, w_2_rr)

mse_1_ols = mse(D1_y_test, y_pred_1_ols)
mse_1_rr = mse(D1_y_test, y_pred_1_rr)

mse_2_ols = mse(D2_y_test, y_pred_2_ols)
mse_2_rr = mse(D2_y_test, y_pred_2_rr)

print(y_pred_1_ols)
print(y_pred_1_rr)

print(f"Mean Squared Error for Linear 1 OLS: {mse_1_ols}")
print(f"Mean Squared Error for Linear 1 Ridge Regression: {mse_1_rr}")
print(f"Mean Squared Error for Linear 2 OLS: {mse_2_ols}")
print(f"Mean Squared Error for Linear 2 Ridge Regression: {mse_2_rr}")

## Stock : BABA

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("BABA.csv")
closing_prices = df['Close'].values

mean = np.mean(closing_prices)
std = np.std(closing_prices)

d = (closing_prices - mean) / std

print(d.shape) # (745,)


In [ ]:
import numpy as np
import cvxopt
import matplotlib.pyplot as plt
cvxopt.solvers.options['show_progress'] = False

def linear_svr_dual(X_train, y_train, C=1.0, epsilon=0.5):
    N = X_train.shape[0]
    
    K = np.dot(X_train, X_train.T)  # Linear Kernel
    P = cvxopt.matrix(np.block([[K, -K], [-K, K]]))
    q = cvxopt.matrix(np.hstack([epsilon + y_train, epsilon - y_train]))
    
    G = cvxopt.matrix(np.vstack([np.eye(2*N),-np.eye(2*N)]))
    
    h = cvxopt.matrix(np.hstack([np.ones(2*N) * C, np.zeros(2*N)]))
    
    A = cvxopt.matrix(np.hstack([np.ones(N), -np.ones(N)]), (1, 2*N))
    b = cvxopt.matrix(0.0)

    # using cvxopt for optimization
    soln = cvxopt.solvers.qp(P, q, G, h, A, b)
    alpha = np.array(soln['x']).flatten()
    
    return alpha[:N] - alpha[N:]

def rbf_kernel(X, gamma):
    dist = np.sum(X**2, 1).reshape(-1, 1) + np.sum(X**2, 1) - 2 * np.dot(X, X.T)
    return np.exp(-gamma * dist)

def kernelized_svr_dual(X_train, y_train, C=1.0, epsilon=0.1, gamma=0.1):
    N = X_train.shape[0]

    # rbf Kernel
    K = rbf_kernel(X_train, gamma)
    
    P = cvxopt.matrix(np.block([[K, -K], [-K, K]]))
    q = cvxopt.matrix(np.hstack([epsilon + y_train, epsilon - y_train]))
    G = cvxopt.matrix(np.vstack([np.eye(2*N),-np.eye(2*N)]))
    h = cvxopt.matrix(np.hstack([np.ones(2*N) * C, np.zeros(2*N)]))
    
    A = cvxopt.matrix(np.hstack([np.ones(N), -np.ones(N)]), (1, 2*N))
    b = cvxopt.matrix(0.0)

    # usinge cvxopt for optimization
    soln = cvxopt.solvers.qp(P, q, G, h, A, b)
    alpha = np.array(soln['x']).flatten()

    return alpha[:N] - alpha[N:]


def linear_svr_prediction(X_train, X_test, y_train, alpha):
    w = np.dot(X_train.T, alpha)
    return np.dot(X_test, w)

def kernel_svr_prediction(X_train, X_test, y_train, alpha, gamma):
    K = np.exp(-gamma * np.sum((X_train[:, None] - X_test) ** 2, axis=2))
    return np.dot(K.T, alpha)


N = len(d)
time = [7,30,90]
gammas = [1.0,0.1,0.01,0.001]

for t in time:
    temp = [d[i:i+t] for i in range(N - t)]
    X = np.array(temp)
    y = d[t:]

    mid = len(X) // 2
    X_train, X_test = X[:mid], X[mid:]
    y_train, y_test = y[:mid], y[mid:]
    
    # call linear svr
    alpha_linear = linear_svr_dual(X_train, y_train)
    y_pred_linear = linear_svr_prediction(X_train, X_test, y_train, alpha_linear)
    rmse_linear = np.sqrt(mse(y_test, y_pred_linear))

    # plot linear svr
    plt.figure(figsize=(10, 5))
    plt.plot(y_test, label="true closing price", color='blue')
    plt.plot(y_pred_linear, label="predicted closing price", color='green')
    plt.xlabel("days")
    plt.ylabel("closing price")
    plt.title(f"Plot for Linear SVR and time = {t}  ")
    plt.legend()
    plt.grid(True)
    plt.show()

    for gamma in gammas:
        alpha_rbf = kernelized_svr_dual(X_train, y_train, gamma)
        y_pred_rbf = kernel_svr_prediction(X_train, X_test, y_train, alpha_rbf, gamma)
        rmse_rbf = np.sqrt(mse(y_test, y_pred_rbf))

        y_avg = np.array([np.mean(y_test[max(0, i - t):i]) for i in range(len(y_test))])

        # plots
        plt.figure(figsize=(10, 5))
        plt.plot(y_test, label="true closing price", color='blue')
        plt.plot(y_pred_rbf, label="predicted closing price", color='green')
        plt.plot(y_avg, label="average price(last t days)", color='red', linestyle='dashed')
        
        plt.xlabel("days")
        plt.ylabel("closing price")
        plt.title(f"graph for (t={t}, γ={gamma})")
        plt.legend()
        plt.grid(True)
        plt.show()